In [3]:
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
import pickle as pk
import numpy as np
from pathlib import Path
base_dir = os.getenv('BASEDIR')

from tqdm import tqdm
tqdm.pandas()

from lightgbm import LGBMClassifier
from itertools import chain, combinations

In [4]:
import pandas as pd
import os 
import re
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.datasets import make_classification

import os
from collections import defaultdict

In [5]:
datasets = ['qanda','ausvotes','socialsense','riot','parler']

In [6]:
performance = {dataset : {} for dataset in datasets}
gt_f1 = {dataset : {} for dataset in datasets}

In [7]:
import itertools
def all_combinations(l):
    return(itertools.product(l, repeat=2))

In [9]:
granularity = '_per_user'
feature = 'use'
ground_truth = 'URL_LR'
for train, test in itertools.chain(all_combinations(datasets)):
    train_path = os.path.join(base_dir,'data','03_processed',train,'ground_truth', train+'_'+ground_truth+'_'+granularity+'.pk')
    if not os.path.exists(train_path):
        continue
    test_path = os.path.join(base_dir,'data','03_processed',test,'ground_truth', test+'_'+ground_truth+'_'+granularity+'.pk')
    if not os.path.exists(test_path):
        continue
#         print(train, test)
    def load_vector(fp):
        with open(fp, 'rb') as rf:
            vec = pk.load(rf)
        return vec

    y_train = load_vector(train_path).values
    mask_train = y_train >= 0

    y_test = load_vector(test_path).values
    mask_test = y_test >= 0
    if any(mask_train):            
        y_train = y_train[mask_train]
        fp_train = os.path.join(base_dir,'data','03_processed',train,'features', train+'_'+feature+'_'+granularity+'.pk')
        X_train = load_vector(fp_train)
        X_train = X_train[mask_train]

        y_test = y_test[mask_test]
        fp_test = os.path.join(base_dir,'data','03_processed',test,'features', test+'_'+feature+'_'+granularity+'.pk')
        X_test = load_vector(fp_test)
        X_test = X_test[mask_test]



        
        if train == test:
            et = LGBMClassifier(n_estimators=100, min_data_in_leaf=500,colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
            skf = StratifiedKFold(n_splits=5, shuffle=True)
            preds = cross_val_predict(et, X_train, y_train, cv=skf, method='predict_proba')
        else:
            et = LGBMClassifier(n_estimators=100, min_data_in_leaf=500,colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
            et.fit(X_train,y_train)
            preds = et.predict_proba(X_test)
        # evaluating the model
        print(train, test)
        print('Area under the ROC Curve:', roc_auc_score(y_test, preds, multi_class='ovo'))

        performance[train][test] = roc_auc_score(y_test, preds, multi_class='ovo')
#             except Exception as e:
#                 print('Failed for:', train, test)

[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
qanda qanda
Area under the ROC Curve: 0.8254557542086816
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
qanda ausvotes
Area under the ROC Curve: 0.6556570338209232
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
qanda socialsen

In [10]:
col_order = datasets
performance_df =  pd.DataFrame(performance).reindex(index=col_order, columns=col_order) 

In [11]:
performance_df

,qanda,ausvotes,socialsense,riot,parler
qanda,0.825456,0.720389,0.695226,0.678167,0.654925
ausvotes,0.655657,0.744018,0.601331,0.585233,0.542794
socialsense,0.653512,0.660694,0.821745,0.636044,0.633620
riot,0.607963,0.609824,0.580140,0.756516,0.617197
parler,0.614514,0.597748,0.561228,0.639871,0.707932


In [13]:
performance_df.to_csv(os.path.join(base_dir, 'data', '04_results', 'cross_dataset_auc.csv'))